In [2]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets.widgets import interact
import pandas as pd
import time

# Metodo Simplex











In [ ]:


num_variable=widgets.IntSlider(min=2, max=8, step=1, value=2)
num_ecuaciones=widgets.IntSlider(min=1, max=5, step=1, value=3)


print("A continuación se haran algunas preguntas sobre su problema. \n")
print("Ingrese el numero de variables en el problema: ")

display(num_variable) 

print("\n ¿Con cuantas ecuaciones cuenta su problema?")
display(num_ecuaciones)

A continuación se haran algunas preguntas sobre su problema. 

Ingrese el numero de variables en el problema en el formato estandar: 


IntSlider(value=2, max=8, min=2)


 ¿Con cuantas ecuaciones cuenta su problema?


IntSlider(value=3, max=5, min=1)

In [ ]:
variables=[f"x{i}" for i in range(1,num_variable.value+1)]
variables.append("b")
nombres_ecuaciones=["Ecuación objetivo"]
nombres_ecuaciones.extend([f"Ecuación {i}" for i in range(1,num_ecuaciones.value+1)])
tabla = pd.DataFrame(0, index=list(range(num_ecuaciones.value+1)), columns=variables)

columns=len(variables)

for i in range(0,num_ecuaciones.value+1):
  for j in range(0,columns):
    while True:
      try:
        
        if(not(i==0 and j==columns-1)):
          coef=float(input(f"\n Ingrese el coeficiente de {variables[j]} para la {nombres_ecuaciones[i]}: "))
          break
        else:
          coef=0
          break
      except ValueError:
        print("Introdujo un valor invalido.")

    tabla.iloc[i,j]=coef


 Ingrese el coeficiente de x1 para la Ecuación objetivo: 1

 Ingrese el coeficiente de x2 para la Ecuación objetivo: 2

 Ingrese el coeficiente de x1 para la Ecuación 1: 3

 Ingrese el coeficiente de x2 para la Ecuación 1: 4

 Ingrese el coeficiente de b para la Ecuación 1: 5

 Ingrese el coeficiente de x1 para la Ecuación 2: 6

 Ingrese el coeficiente de x2 para la Ecuación 2: 7

 Ingrese el coeficiente de b para la Ecuación 2: 8

 Ingrese el coeficiente de x1 para la Ecuación 3: 9

 Ingrese el coeficiente de x2 para la Ecuación 3: 4

 Ingrese el coeficiente de b para la Ecuación 3: 7


In [ ]:
tabla

,x1,x2,b
0,1,2,0
1,3,4,5
2,6,7,8
3,9,4,7


In [ ]:
simbolos=["="]
options=['<', '<=', '>','>=','=']
print("Ingrese '<=', '<', '>', '>=' o '=' correspondiente a cada ecuación.")
for i in range(1, num_ecuaciones.value+1):
  while True:
    try:
      simbol=input(f"\n ¿Con que simbolo cuenta la ecuación {i}?  ")
      if (simbol in options):
        break
      else:
        raise ValueError()
    except ValueError:
        print("Introdujo un valor invalido.")
  simbolos.append(simbol)

simbolos

tabla["simbolo"]=simbolos


Ingrese '<=', '<', '>', '>=' o '=' correspondiente a cada ecuación.

 ¿Con que simbolo cuenta la ecuación 1?  <

 ¿Con que simbolo cuenta la ecuación 2?  <

 ¿Con que simbolo cuenta la ecuación 3?  <


In [ ]:
tabla=tabla[["x1","x2", "simbolo", "b"]]
tabla

,x1,x2,simbolo,b
0,1,2,=,0
1,3,4,<,5
2,6,7,<,8
3,9,4,<,7


Usando la libreria de Scipy y la función linprog podemos solucionar el problema. 


La libreria esta hecha para soluccionar el siguiente problema:

$$ min \;C^{T}\cdot X$$

Si queremos maximizar, multiplicaremos al vector $C$ por $(-1)$.

Existen dos formas de introduccir el problema:

$$\textbf{Forma estandar}$$

$$A_{ub} \cdot X \leq b_{ub}$$

$$\textbf{Forma aumentada}$$

$$A_{eq} \cdot X = b_{eq}$$

Nota: La condición de no negatividad ya viene por defecto.


In [ ]:


import numpy as np
from scipy.optimize import linprog

c = np.array([-3, -5]) #El problema es max z = 3x_1+5x_2, pero como la libreria pide min, entonces se multiplica todo por -1.
A = np.array([[1, 0],
              [0, 2],
              [3, 2]])
b = np.array([4, 12, 18])
x0_bounds = (0, None) #Pone la condición de no negatividad. (0,inf) Nota: No es ponerlos si es (0, inf)
x1_bounds = (0, None) #Pone la condición de no negatividad. (0,inf) Nota: No es ponerlos si es (0, inf)

res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds],method="highs" )
print(res.x)
print(res.fun)
print(res.slack)
print(res.con)


[2. 6.]
-36.0
[2. 0. 0.]
[]


In [ ]:
import numpy as np
from scipy.optimize import linprog

c = np.array([-3, -5,0,0,0]) #El problema es max z = 3x1+5x2, pero como la libreria pide min, entonces se multiplica todo por -1.
A = np.array([[1, 0,1,0,0],
              [0, 2,0,1,0],
              [3, 2,0,0,1]])
b = np.array([4, 12, 18])
res = linprog(c, A_eq=A, b_eq=b, method="highs")
print(res.x)
print(res.fun)
print(res.slack)
print(res["con"])

[2. 6. 2. 0. 0.]
-36.0
[]
[0. 0. 0.]


Todavia nos falta hacer el codigo para que muestre las diferentes tablas.
Desde aqui se intentara.

## Analizando codigo

In [ ]:
import numpy as np
from scipy.optimize import linprog

c = np.array([3, 5])
A = np.array([[1, 0], 
              [0, 2], 
              [3, 2]])
b = np.array([4, 12, 18])

B = np.array([])  # Inicializar la base vacía

res = linprog(c, A_ub=A, b_ub=b, method='revised simplex')
x = res['x']
B = A[:, x > 0]  # Actualizar la base
N = A[:, x == 0]  # Actualizar la no-base
z = np.dot(c, x)

print(f"Iteración 0:\n x = {x}\n z = {z}\n B =\n {B}\n N =\n {N}\n")

i = 1
while res['status'] == 0:
    cB = c[x > 0]  # Seleccionar los coeficientes de c correspondientes a la base
    cN = c[x == 0]  # Seleccionar los coeficientes de c correspondientes a la no-base
    Binv = np.linalg.inv(B)  # Calcular la inversa de la base
    y = np.dot(cB, Binv)  # Calcular los multiplicadores simplex
    sN = cN - np.dot(y, N)  # Calcular los costos reducidos
    j = np.argmax(sN)  # Determinar el índice de la variable a ingresar a la base
    d = np.dot(Binv, A[:, j])  # Calcular la dirección simplex
    if np.all(d <= 0):
        print("El problema no tiene solución acotada.")
        break
    else:
        xB = np.dot(Binv, b)  # Calcular las variables básicas
        theta = xB / d[x > 0]  # Calcular los pasos admisibles
        i = np.argmin(theta)  # Determinar el índice del paso a tomar
        x[x > 0] = xB - theta[i] * d[x > 0]  # Actualizar las variables básicas
        x[j] = theta[i]  # Ingresar la nueva variable a la base
        B = A[:, x > 0]  # Actualizar la base
        N = A[:, x == 0]  # Actualizar la no-base
        z = np.dot(c, x)  # Calcular el valor objetivo
        print(f"Iteración {i}:\n x = {x}\n z = {z}\n B =\n {B}\n N =\n {N}\n")
        res = {'status': 0}  # Actualizar el estado para continuar el ciclo


Iteración 0:
 x = [0. 0.]
 z = 0.0
 B =
 []
 N =
 [[1 0]
 [0 2]
 [3 2]]



<ipython-input-101-ba602636458b>:10: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_ub=A, b_ub=b, method='revised simplex')


LinAlgError: ignored